# get Cause - efffect phrase from the sentence.

In [1]:
#filtered_rows.reset_index(drop=True, inplace=True)


## 1. load cause-effect data

In [2]:
#load CE semeval dataset
import pandas as pd
sem_CE_test= pd.read_csv('data/in/RAG/c_e_Extraction/SCITE_CE_extract_INPUT.csv')
pm_DB = sem_CE_test
pm_DB

,Unnamed: 0,sentence
0,0,Using solar electricity instead of conventiona...
1,1,"Thus, evaluating capital punishment as a form ..."
2,2,Sudden death from inhalation of petroleum ...
3,3,Stimulation of the posterior horn of the med...
4,4,Sip the tea slowly to reduce stomach pain fr...
...,...,...
186,186,The disruption has been caused by work bei...
187,187,"Most illnesses, including colds and flu , c..."
188,188,Every child and young person seen at Starship ...
189,189,The movie gives the inaccurate impression ...


## 2. load few shot database

In [3]:
#load semeval sentence and patterns.
semeval_data = pd.read_csv('data/in/RAG/comb_SCITEsemADE_CausalityPattern.csv')# 

semeval_data = semeval_data.dropna(subset=['causality_phrase'])
len(semeval_data)

#semeval_data = pd.read_csv('data/in/semeval_CE_pattern.csv')# data/in/ADE/ADE_CE_pattern.csv
causality_phrase = semeval_data['causality_phrase'].to_list()
len(causality_phrase)

2365

## 3. identify cause- effect phrase

In [4]:
#load causality input sentence.
#input_sentence_list = filtered_rows['sentence'].to_list()
#gpt to extract cause effect and a causality phrase.
#: few shot: example1: , causality phrase: .

In [5]:
import groq
from groq import Groq

client = groq.Groq(api_key= "enter your API key here..")#my iu mail key

#gemma2-9b-it# llama3-8b-8192
#mixtral-8x7b-32768
def C_E_CP_Identifier(causality_sentence, fewshot_examples):

    chat_completion = client.chat.completions.create(
    #
    # Required parameters
    #
    messages=[
        # Set an optional system message. This sets the behavior of the
        # assistant and can be used to provide specific instructions for
        # how it should behave throughout the conversation.
        {
          "role": "system",
          "content": '''
  
          You will be given a sentence that shows a causality relation. 
          Your task is to identify all the cause phrase, effect phrase, and causality phrase from
          that sentence. Below is the hints and rules for the task.

          Hints: In sentences showing a causality relation, you can identify a pair of phrases
          where one phrase causes the other (effect). There will also be a phrase between the
          cause and effect phrases that indicates the causality relation; this is the 
          causality phrase. And in most sentences causality phrase can be seen between the 
          cause and effect phrase.
           

          Your output should be in JSON format with keys cause, effect, and causality_phrase:
          {"cause1": "", "effect1": "", "causality_phrase1": "", "cause2": "", "effect2": "", "causality_phrase2": "",
          "cause3": "", "effect3": "", "causality_phrase3": "", ....}. Make sure your JSON ouput is closed with } at the end.
          
          Rule 1: If you are not able to find the cause phrase, effect phrase, and causality phrase 
          from the given sentence you can output empty string for the keys in the json format 
          and do not guess the answer. 
          
          Rule 2: Ensure that there are no overlapping phrases between the cause phrase, effect phrase,
          and causality phrase. For instance, consider the input sentence "Fever is caused by virus." 
          Your identified output without overlap should be:
          {"cause": "virus", "effect": "fever", "causality_phrase": "caused by"}
          Your output should not contain overlap, as shown below:
          {"cause": "by virus", "effect": "fever is caused", "causality_phrase": "caused by"}
          This means that each phrase (cause, effect, and causality phrase) should be distinct 
          and not overlap with each other.
          
          Rule 3: There could be multiple cause-effect phrases in the sentence. You should extract all 
          of them. For instance, consider the input sentence "Mutations in MCFD2 causing F5F8D are null
          mutations but also missense mutations, which cause destabilization of its flexible structure
          and the loss of its interaction with ERGIC-53 .". 
          Your identified output will be:
          {"cause1": "Mutations in MCFD2", "effect1": "F5F8D", "causality_phrase1": "causing", 
          "cause2": "missense mutations", "effect2": "destabilization of its flexible structure", "causality_phrase2": "cause",
          "cause3": "missense mutations", "effect3": "the loss of its interaction with ERGIC-53", "causality_phrase3": "cause"

          
          Below are 10 example sentences that show causality relations, with their cause and
          effect phrases delimited between the XML tags <cause>, </cause>, <effect>, and 
          </effect>. The corresponding causality phrases are also provided.
          

                   
          ''' + fewshot_examples
            
        },
        {
          "role": "user",
          "content": input_sentence 
        },
    ],

    # The language model which will generate the completion.
    model="mixtral-8x7b-32768",
    temperature=0.0,
    seed = 1,
    max_tokens=100,
    top_p=1,
    response_format={"type": "json_object"},)
    out = chat_completion.choices[0].message.content
    
    return out

In [6]:
def get_fewshotexamples_C_E_CP(fewshot_index):

    
    #fewshot_index = [1, 5, 8, 10, 30, 100, 102, 105, 109, 120]
    
    
    train_sentence = semeval_data['sentence'].to_list()
    train_causalityPhrase = semeval_data['causality_phrase'].to_list()

    fewshot_example_sentence = [train_sentence[i] for i in fewshot_index]
    fewshot_example_causalityPhrase = [train_causalityPhrase[i] for i in fewshot_index]


    import json
    #fewshot_examples = {f'example {i + 1}': sentence for i, sentence in enumerate(fewshot_examples)}
    fewshot_examples = {f'example {i + 1}': {'sentence': sentence, 'causalityPhrase': fewshot_example_causalityPhrase[i]} 
                    for i, sentence in enumerate(fewshot_example_sentence)}
    
    # Assuming json_data is already defined
    fewshot_examples = json.dumps(fewshot_examples)
    #print(fewshot_examples)
    
    return fewshot_examples

In [7]:
# semeval
filtered_rows = pm_DB

In [8]:
#pattern matcher function.
import random
from fuzzywuzzy import fuzz

def get_fewshot_examples_index(input_sentence) :

    #get few shot example index using Leiveinstein ratio.

    fewshot_index = []
    for ind in range (len(causality_phrase)):
        p = causality_phrase[ind]
        #print('ind :', ind)
        window_size = len(p)

        # Slide through input_sentence with a window size 
        for i in range(len(input_sentence) - window_size + 1):
            window = input_sentence[i:i + window_size]
            score = fuzz.ratio(window, p)
            if score >90:
                fewshot_index.append(ind)
                #print('causality phrase: ',p, 'index is', ind)
                #print(f"Window '{window}' - LS: {score}")

    #print('fewshot_index', fewshot_index)
    fewshot_index = list(set(fewshot_index))
    #print('fewshot_index', fewshot_index)
    
    if len(fewshot_index)>10:
        fewshot_index = random.sample(fewshot_index, 10)
        
    
    return fewshot_index
    

#get all sentences and save as json example1

In [9]:
import json
import re

def close_json_string(json_string):
    
    
    # Remove entries where "cause", "effect", and "causality_phrase" are all empty
    pattern = re.compile(r'"cause\d+":\s*"",\s*"effect\d+":\s*"",\s*"causality_phrase\d+":\s*""\s*,?\s*')
    json_string = re.sub(pattern, '', json_string)
    
    # Remove incomplete index entries
    incomplete_index_pattern = re.compile(r'"cause\d+":\s*""\s*,?\s*')
    json_string = re.sub(incomplete_index_pattern, '', json_string)
    
    # Remove trailing comma if exists
    json_string = re.sub(r',\s*}$', '}', json_string.strip())
    json_string = re.sub(r',\s*$', '', json_string.strip())
    
    
    
    # Find the last complete index
    last_index = re.findall(r'"cause\d+":\s*"[^"]*",\s*"effect\d+":\s*"[^"]*",\s*"causality_phrase\d+":\s*"[^"]*"', json_string)
    
    # If found, truncate the JSON string after the last complete index
    if last_index:
        last_index_end_pos = json_string.rfind(last_index[-1]) + len(last_index[-1])
        json_string = json_string[:last_index_end_pos]
    
    # Count the number of opening and closing braces
    open_braces = json_string.count('{')
    close_braces = json_string.count('}')
    
    # Add missing closing braces if needed
    if open_braces > close_braces:
        json_string += '}' * (open_braces - close_braces)
    
    return json_string

In [10]:
import json
sent_cause_effect_list = []
causality_phrase_list = []

fewshotfound_inputsentence_index=[]
for i in range(len(filtered_rows)):
    input_sentence = filtered_rows['sentence'][i]
    #input_sentence = 'fever is caused by virus'
    
    #print('input_sentence', input_sentence)
    
    # few shot examples augmentation.
    fewshot_index = get_fewshot_examples_index(input_sentence)#########FEWSHOT
    #print('fewshot_index before :', fewshot_index)
    ##
    if len(fewshot_index) == 0:
        fewshot_index = [k for k in range(10)]
        fewshot_examples =  get_fewshotexamples_C_E_CP(fewshot_index) 
    else:
        fewshotfound_inputsentence_index.append(i)
        
        #fewshot_index = [k for k in range(10)]###############################RANDOMSHOT/zero
        fewshot_examples =  get_fewshotexamples_C_E_CP(fewshot_index) 
        
    ##
    print('fewshot_index :', fewshot_index)
    ##############print('fewshot_examples', fewshot_examples)

    #fewshot_examples = get_fewshotexamples_C_E_CP(fewshot_index) 
    
    #print('fewshotfound_inputsentence_index', fewshotfound_inputsentence_index)
    try:
    
        # output generation
        out = C_E_CP_Identifier(input_sentence, fewshot_examples)
        #print('out1 : ', out)

        out = close_json_string(out)
        print('out : ', out)

    
    # Attempt to parse the JSON string into a dictionary
        data = json.loads(out)
    except json.JSONDecodeError as e:
        print("Failed to parse JSON:", e)
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])
        continue  # Skip to the next iteration if there is a JSON error

        
    except TypeError as e:
        print('TypeError')
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])
        continue
        
    except KeyError as e:
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])
        continue
    
    except groq.APIConnectionError as e:
        print('APIConnectionError')
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])
        continue
        
    except groq.RateLimitError as e:
        print('RateLimitError')
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])    
        continue
        
    except groq.APIStatusError as e:
        print('APIStatusError')
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])
        continue
    # Parse JSON string into a dictionary
    data = json.loads(out)

    # Initialize lists
    sent_cause_effect = []
    causality_phrase_row = []

    # Iterate through the keys and collect values
    i_out = 1
    while f'cause{i_out}' in data and f'effect{i_out}' in data and f'causality_phrase{i_out}' in data:
        sent_cause_effect.append((input_sentence, data[f'cause{i_out}'], data[f'effect{i_out}']))
        causality_phrase_row.append(data[f'causality_phrase{i_out}'])
        i_out += 1


    if sent_cause_effect == []:
        print('empty')
        try:
            sent_cause_effect.append((input_sentence, data['cause'], data['effect']))
            causality_phrase_row.append(data['causality_phrase'])

        except KeyError as e:
            print('key...')
            sent_cause_effect_list.append([])
            causality_phrase_list.append([])
            continue        

        
    # Print the results
    print("sent_cause_effect:", sent_cause_effect)
    #print("causality_phrase_row:", causality_phrase_row)
    sent_cause_effect_list.append(sent_cause_effect)
    causality_phrase_list.append(causality_phrase_row)
    print('i:', i)
    
    #if i==3:
        #break;
    

fewshot_index : [523, 2093, 1136, 1510, 2017, 1531, 1511, 1506, 280, 2007]
out :  {
  "cause1": "using solar electricity instead of conventional electricity sources",
  "effect1": "reducing pollution",
  "causality_phrase1": "helps to",
  "cause2": "reducing pollution",
  "effect2": "that causes smog, acid rain, respiratory illness and global warming",
  "causality_phrase2": "that"}
sent_cause_effect: [('Using solar electricity instead of conventional electricity sources helps to reduce  pollution  that causes  smog ,  acid rain ,  respiratory illness  and  global warming .', 'using solar electricity instead of conventional electricity sources', 'reducing pollution'), ('Using solar electricity instead of conventional electricity sources helps to reduce  pollution  that causes  smog ,  acid rain ,  respiratory illness  and  global warming .', 'reducing pollution', 'that causes smog, acid rain, respiratory illness and global warming')]
i: 0
fewshot_index : [1506, 2089, 481, 2086, 1809, 1

fewshot_index : [1533, 1509, 1532, 1512, 1530, 523, 1518, 520, 521, 1529]
APIStatusError
fewshot_index : [1816, 1821, 1813, 1815, 2092, 1819, 1814, 1817, 1820, 1551]
out :  {
  "cause": "the storm",
  "effect1": "31 deaths",
  "causality_phrase1": "resulted in",
  "effect2": "2.5 million dollars damage",
  "causality_phrase2": "resulted in"
}
empty
key...
fewshot_index : [1510, 548, 1136, 522, 519, 520, 1515, 1532, 1530, 555]
out :  {
  "cause": "a fuel leak in one of the C-17s",
  "effect": "a delay",
  "causality_phrase": "caused"
}
empty
sent_cause_effect: [(' A fuel leak  in one of the C-17s caused  a delay  on the morning of Nov. 7 as the crew was preparing for departure for Cairo, Egypt.', 'a fuel leak in one of the C-17s', 'a delay')]
i: 19
fewshot_index : [1241, 519, 1532, 1833, 557, 523, 1531, 1527, 1533, 549]
out :  {
  "cause1": "steam",
  "effect1": "rupture disc opening",
  "causality_phrase1": "caused a backpressure on the VTD, resulting in"}
sent_cause_effect: [(' The st

out :  {"cause1": "hormone deficiencies and imbalances", "effect1": "Paralysis or convulsions", "causality_phrase1": "are caused by"}
sent_cause_effect: [(' Paralysis  or  convulsions  are caused by  hormone deficiencies and imbalances .', 'hormone deficiencies and imbalances', 'Paralysis or convulsions')]
i: 36
fewshot_index : [993, 996, 1699, 999, 2238, 995, 2092, 2091, 1513, 1511]
out :  {
  "cause": "the fireplace",
  "effect": "warmth",
  "causality_phrase": "radiating from"
}
empty
sent_cause_effect: [(' The warmth  was radiating from  the fireplace  to all corners of the room.', 'the fireplace', 'warmth')]
i: 37
fewshot_index : [517, 550, 2205, 2200, 594, 556, 1136, 477, 520, 595]
out :  {"cause": "the asteroid that passed Earth yesterday", "effect": "the boom and shaking", "causality_phrase": "was caused by"}
empty
sent_cause_effect: [(' The boom  and  shaking  was caused by  the asteroid  that passed Earth yesterday.', 'the asteroid that passed Earth yesterday', 'the boom and 

fewshot_index : [1513, 1241, 1529, 520, 439, 440, 278, 1512, 442, 445]
out :  {
  "cause1": "The debris",
  "effect1": "a crash on the Schuylkill",
  "causality_phrase1": "caused",
  "cause2": "the mishap",
  "effect2": "it has been closed for hours",
  "causality_phrase2": "because of"}
sent_cause_effect: [(' The debris  caused  a crash  on the Schuylkill, and it has been  closed  for hours because of  the mishap .', 'The debris', 'a crash on the Schuylkill'), (' The debris  caused  a crash  on the Schuylkill, and it has been  closed  for hours because of  the mishap .', 'the mishap', 'it has been closed for hours')]
i: 57
fewshot_index : [280, 519, 478, 1527, 1136, 521, 1508, 479, 1242, 1512]
out :  {
  "cause1": "Alcohol and drugs",
  "effect1": "suicide",
  "causality_phrase1": "directly cause"}
sent_cause_effect: [(' Alcohol  and  drugs  directly cause  suicide  by significantly diminishing the reasoning of the person at the time of the suicide.', 'Alcohol and drugs', 'suicide')]


fewshot_index : [1515, 1318, 279, 1509, 122, 127, 1511, 1514, 278, 1507]
APIStatusError
fewshot_index : [1518, 1531, 1512, 1506, 1814, 1507, 1514, 1533, 1816, 1528]
out :  {
  "cause": "the crashes",
  "effect": "the collapse of much of the World Trade Center complex, the destruction of part of the southwest side of the Pentagon",
  "causality_phrase": "resulted in"
}
empty
sent_cause_effect: [(' The crashes  resulted in  the collapse  of much of the World Trade Center complex,  the destruction  of part of the southwest side of the Pentagon.', 'the crashes', 'the collapse of much of the World Trade Center complex, the destruction of part of the southwest side of the Pentagon')]
i: 77
fewshot_index : [1318, 1551, 1136, 625, 626, 627, 628]
out :  {
  "cause": "germs",
  "effect1": "flu",
  "causality_phrase1": "killed",
  "effect2": "colds",
  "causality_phrase2": "killed",
  "effect3": "sore throat",
  "causality_phrase3": "killed"
}
empty
key...
fewshot_index : [2093, 1136, 1507, 2091,

fewshot_index : [1528, 522, 514, 523, 985, 553, 595, 515, 551, 478]
out :  {
  "cause": "eating raw or undercooked food",
  "effect": "infection caused by the bacteria called Salmonella",
  "causality_phrase": "by"
}
empty
sent_cause_effect: [('General information about  the infection  caused by  the bacteria  called Salmonella, when food is eaten raw or undercooked.', 'eating raw or undercooked food', 'infection caused by the bacteria called Salmonella')]
i: 97
fewshot_index : [992, 1000, 279, 816, 811, 994, 1533, 2093, 2094, 280]
out :  {
  "cause": "breathing small objects",
  "effect": "suffocation",
  "causality_phrase": "due to"
}
empty
sent_cause_effect: [('Each year, many infants and toddlers  die  due to  suffocation  from  breathing small objects  into their breathing passages and lungs.', 'breathing small objects', 'suffocation')]
i: 98
fewshot_index : [1510, 515, 1513, 1256, 548, 549, 595, 2094, 1263, 1241]
out :  {
  "cause": "the bacterial toxin",
  "effect": "muscle para

out :  {
  "cause": "widgets resembling HTC's own Sense UI widgets",
  "effect": "the problem",
  "causality_phrase": "comes from"
}
empty
sent_cause_effect: [(" The problem  comes from  the widgets  resembling HTC's own Sense UI widgets.", "widgets resembling HTC's own Sense UI widgets", 'the problem')]
i: 122
fewshot_index : [280, 522, 519, 514, 1529, 1136, 520, 516, 515, 1532]
out :  {
  "cause1": "Liver",
  "effect1": "diabetes and hypertension",
  "causality_phrase1": "how steroids cause"}
sent_cause_effect: [('Liver plays key role in how  steroids  cause  diabetes  and  hypertension .', 'Liver', 'diabetes and hypertension')]
i: 123
fewshot_index : [985, 902, 1528, 1241, 1529, 1242, 1532, 280, 278, 279]
out :  {"cause": "Zinc", "effect1": "growth", "effect2": "cell division", "causality_phrase": "essential for"}
empty
key...
fewshot_index : [1023, 1532, 1001, 2087, 1529, 2094, 1528, 1533, 1243, 2086]
out :  {
  "cause": "the atomic bomb explosion",
  "effect": "radiation",
  "caus

fewshot_index : [592, 598, 481, 594, 123, 1511, 554, 2086, 549, 556]
out :  {
  "cause": "the tornado",
  "effect": "the damage",
  "causality_phrase": "by examining the damage caused by the tornado"
}
empty
sent_cause_effect: [('The Fujita Scale is used to rate the intensity of a tornado by examining  the damage  caused by  the tornado  after it has passed over a man-made structure.', 'the tornado', 'the damage')]
i: 144
fewshot_index : [1528, 279, 518, 517, 1551, 1530, 280, 1241, 2086, 1529]
out :  {"cause1": "Various hormonal, bacterial and inflammatory disturbances", "effect1": "acne", "causality_phrase1": "cause"}
sent_cause_effect: [(' Various hormonal, bacterial and inflammatory disturbances  cause  acne .', 'Various hormonal, bacterial and inflammatory disturbances', 'acne')]
i: 145
fewshot_index : [557, 553, 549, 1533, 1506, 1528, 523, 278, 1508, 518]
out :  {
  "cause1": "the earthquake",
  "effect1": "the failures of the electric power system, the water supply system, the se

out :  {"cause": "this disease", "effect": "pain", "causality_phrase": "from", "cause2": "adenomyosis", "effect2": "experience", "causality_phrase2": "experience"}
empty
sent_cause_effect: [("Patients with  adenomyosis experience pain  from  this disease  in the early 30's.", 'this disease', 'pain')]
i: 162
fewshot_index : [1383, 1320, 1384, 487, 1385, 1386, 1387, 1388, 1136]
out :  {"cause": "the cow", "effect": "a sound called lowing, also known as mooing", "causality_phrase": "makes"}
empty
sent_cause_effect: [(' The cow  makes  a sound  called lowing, also known as mooing.', 'the cow', 'a sound called lowing, also known as mooing')]
i: 163
fewshot_index : [996, 1272, 998, 1136, 1242, 999, 993, 994, 1271, 992]
out :  {"cause": "the sunrise", "effect": "The light in the background", "causality_phrase": "from"}
empty
sent_cause_effect: [(' The light  in the background is from  the sunrise .', 'the sunrise', 'The light in the background')]
i: 164
fewshot_index : [612, 324, 2088, 1510, 

fewshot_index : [994, 1527, 999, 279, 1242, 1531, 1529, 1241, 938, 482]
out :  {
  "cause": "indigestion",
  "effect": "chest pain",
  "causality_phrase": "from",
  "additional_info1": "feels like an intense burning",
  "additional_info2": "is usually accompanied by regurgitation and gas"
}
empty
sent_cause_effect: [(' Chest pain  from  indigestion  feels like an intense burning and is usually accompanied by regurgitation and gas.', 'indigestion', 'chest pain')]
i: 180
fewshot_index : [1647, 1643, 1507, 1649, 1136, 2340, 2334, 1636, 1637, 1641]
out :  {
  "cause": "the Shia ceasefire",
  "effect": "the relative calm",
  "causality_phrase": "produced by"
}
empty
sent_cause_effect: [(' The relative calm  produced by  the Shia ceasefire  has coincided with what the CIA is now calling the "near strategic defeat" of al-Qaeda in Iraq.', 'the Shia ceasefire', 'the relative calm')]
i: 181
fewshot_index : [522, 2086, 479, 596, 549, 482, 597, 1528, 520, 595]
out :  {
  "cause1": "the asteroid",


In [11]:
len(filtered_rows)

191

In [12]:
#filtered_rows = filtered_rows.drop('GPT_causality_label', axis = 1)
#filtered_rows = filtered_rows.drop('patternmatcher_label', axis = 1)
len(fewshotfound_inputsentence_index)

191

In [13]:
filtered_rows['sent_cause_effect'] = sent_cause_effect_list
filtered_rows['causality_phrase'] = causality_phrase_list
filtered_rows

,Unnamed: 0,sentence,sent_cause_effect,causality_phrase
0,0,Using solar electricity instead of conventiona...,[(Using solar electricity instead of conventio...,"[helps to, that]"
1,1,"Thus, evaluating capital punishment as a form ...","[(Thus, evaluating capital punishment as a for...",[that result in]
2,2,Sudden death from inhalation of petroleum ...,[( Sudden death from inhalation of petroleu...,[from]
3,3,Stimulation of the posterior horn of the med...,[( Stimulation of the posterior horn of the m...,[produced]
4,4,Sip the tea slowly to reduce stomach pain fr...,[],[]
...,...,...,...,...
186,186,The disruption has been caused by work bei...,[( The disruption has been caused by work b...,[has been caused by]
187,187,"Most illnesses, including colds and flu , c...","[(Most illnesses, including colds and flu ,...","[cause, that also]"
188,188,Every child and young person seen at Starship ...,[(Every child and young person seen at Starshi...,[with]
189,189,The movie gives the inaccurate impression ...,[( The movie gives the inaccurate impression...,[gives]


In [14]:
#SCITE

pm_DB = pd.read_csv('data/in/RAG/c_e_Extraction/SCITE_CE_extract_INPUTkeys.csv')
pm_DB['keys'] = pm_DB['keys'].apply(eval)



C_t = pm_DB['keys'].to_list()


C_p = sent_cause_effect_list

In [15]:
len(C_t)

296

In [16]:
len(sent_cause_effect_list)

191

In [17]:
C_p = [tup for sublist in C_p for tup in sublist]
#len(C_p[1])


In [18]:
def get_score(true_phrase, pred_phrase ):
    scores=[]
    p = true_phrase
    window_size = len(p)

        # Slide through input_sentence with a window size of 8
    if (pred_phrase) =="":
        print('no pred_phrase')
        return 0
    else: 
        for i in range(len(pred_phrase) - window_size + 1):
            window = pred_phrase[i:i + window_size]
            score = fuzz.ratio(window, p)
            scores.append(score)
        if len(scores) ==0:
            print('prediction length lower', pred_phrase, 'p:', p)
            return 0
        else:
            return max(scores)

In [19]:
len(C_p)

191

In [20]:
prediction= []
pred_label = []
for i in range(len(C_p)):
    #print(i)
    labeled = 0   
    if (C_p[i] != []):
        if len(C_p[i]) == 3:           
            
            sent = C_p[i][0]
            ####print('sent: ',sent )
            cause_pred = C_p[i][1]
            effect_pred = C_p[i][2]
            
            for i_t in range(len(C_t)):
                #get index of all tuples which match the sent and create sent_found = []
                # List to store indices
                sent_found = []

                # Check if sent is in the C_t at index 0
                for index, (sentence, _, _) in enumerate(C_t):
                    if sentence == sent:
                        sent_found.append(index)
            ####print('sent_found: ', sent_found)
            
            
            #iterate through sent_found and pass each cause_t and effect_t to get score: 
            for ind in sent_found:
                cause_t = C_t[ind][1]
                effect_t = C_t[ind][2]
                pred_c=0
                pred_e=0
                ##print('cause_t: ', cause_t)
                ##print('effect_t: ', effect_t)
                ##print('cause_pred: ', cause_pred)
                ##print('effect_pred: ', effect_pred)
                
                score = get_score(cause_t.lower(), cause_pred.lower())################ non -medcause
                    #score = fuzz.ratio(cause_pred, cause_t)################medcause
                if score >90:################90 non -medcause
                    pred_c = 1
                else:
                    pred_c=0
                    


                score = get_score(effect_t.lower(), effect_pred.lower())################non - medcause
                    #score = fuzz.ratio(effect_pred, effect_t)################medcause

                if score >90: ################90 non -medcause
                    pred_e = 1
                else:
                    pred_e=0



                if ((pred_e ==1) & (pred_c ==  1)):
                    prediction.append(1)
                    
                    labeled = 1
                    #pred_label.append(labeled)
                    #print('out:', 1)
    
    
    
    
    pred_label.append(labeled)
                    
    print(labeled, i)



                
    

 

prediction length lower reducing pollution p: respiratory illness
0 0
1 1
0 2
1 3
1 4
1 5
0 6
0 7
1 8
1 9
1 10
1 11
prediction length lower heat p: smoke
1 12
prediction length lower wind p: smoke
1 13
1 14
1 15
1 16
0 17
prediction length lower headache p: a headache
0 18
0 19
1 20
prediction length lower steam p: the steam
prediction length lower steam p: a backpressure
prediction length lower rupture disc opening p: the rupture disc opening
0 21
1 22
1 23
0 24
1 25
1 26
1 27
prediction length lower oak root and chamomile p: environmental pollutants
0 28
0 29
prediction length lower the drone strike p: deliberately planted false intelligence
1 30
1 31
1 32
1 33
1 34
1 35
prediction length lower this feeling p: helping others
prediction length lower this feeling p: helping others
0 36
1 37
1 38
1 39
prediction length lower warmth p: the warmth
0 40
1 41
1 42
1 43
1 44
prediction length lower nuclear energy p: even nuclear energy
0 45
prediction length lower smog, acid rain p: other br

In [21]:
prec=sum(prediction)/len(C_p)
rec = sum(prediction)/len(C_t)
F1 = 2 *(prec * rec)/(prec + rec)
print('F1: ', F1)
print('prec: ', prec)
print('rec: ', rec)


F1:  0.6735112936344969
prec:  0.8586387434554974
rec:  0.5540540540540541


In [22]:
len(pred_label)

191

In [23]:
len(C_p)

191

In [24]:
SCITE_df = pd.DataFrame({'pred_label': pred_label, 'C_p' : C_p})
SCITE_df

,pred_label,C_p
0,0,(Using solar electricity instead of convention...
1,1,(Using solar electricity instead of convention...
2,0,"(Thus, evaluating capital punishment as a form..."
3,1,( Sudden death from inhalation of petroleum...
4,1,( Stimulation of the posterior horn of the me...
...,...,...
186,1,"(Most illnesses, including colds and flu , ..."
187,0,"(Most illnesses, including colds and flu , ..."
188,0,(Every child and young person seen at Starship...
189,1,( The movie gives the inaccurate impression ...


In [25]:
#filtered_rows.to_csv('data/out/RAG_C_E_CP_Alzeimers.csv')#data/out/RAG_causality/semeval/semeval_
#filtered_rows.to_csv('data/out/RAG_causality/semeval/fewshot_sem_CE_detection.csv')#data/out/RAG_causality/semeval/semeval_

#fewshot_semeval_CE_detection
#SCITE_df.to_csv('data/out/RAG_causality/task2/gemma/RAG_SCITE_CE_extract.csv')
#filtered_rows.to_csv('data/out/RAG_causality/task2/GPT4/zero_sem_CE_extract.csv')#zero,rand,RAG|sem,ADE,SCITE


In [26]:
#filtered_rows= filtered_rows[filtered_rows['predictions'] == 0]
#filtered_rows.to_csv('data/out/RAG_causality/semeval/zero_sem_RAG_C_E_pred0.csv')#data/out/RAG_causality/semeval/semeval_
